In [254]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy
from pathlib import Path
import nltk as ntlk

In [255]:
# Setup - Load the SQL extension and connect to the Mini IMDB dataset we've prepared
db_path = Path('/Users/ryan/Downloads/biosample_basex_data_good_subset.db')

engine = sqlalchemy.create_engine(f"sqlite:///{db_path}")
connection = engine.connect()
inspector = sqlalchemy.inspect(engine)

#Get table names
inspector.get_table_names()

['bp_id_accession',
 'env_package_repair',
 'harmonized_wide_sel_envs',
 'non_attribute_metadata_sel_envs']

In [256]:
#query_non_attribute_metadata_sel_envs = """
#SELECT * 
#FROM non_attribute_metadata_sel_envs
#"""
#non_attribute_metadata_sel_envs = pd.read_sql(query_non_attribute_metadata_sel_envs, engine)
#non_attribute_metadata_sel_envs

In [257]:
query_harmonized_wide_sel_envs = """
SELECT * 
FROM harmonized_wide_sel_envs
"""
harmonized_wide_sel_envs = pd.read_sql(query_harmonized_wide_sel_envs, engine)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 2000)
rto_column = harmonized_wide_sel_envs.loc[:, 'rel_to_oxygen'].to_frame().head(15)
harmonized_wide_sel_envs.sort_values(['rel_to_oxygen']).loc[:, 'rel_to_oxygen'].to_frame().head(15)
harmonized_wide_sel_envs['rel_to_oxygen'].unique()

array([None, 'facultative anaerobe', 'aerobe', 'obligate', 'anaerobe',
       'oxic', 'hypoxic', 'Aerobic', 'aerobic', 'aerobic-anaerobic',
       'facultative anaerobes', 'oxic/anoxic boundary', 'microaerophilic',
       '1.0-2.2 mg/l', '0 mg/l', '6.0-6.5 mg/l', '22 mg/l', '23.5 mg/l',
       '25 mg/l', 'normal oxic seawater', 'anaerobic', 'Hypoxic',
       'obligate anaerobe', 'Oxic', 'facultative'], dtype=object)

In [258]:
harmonized_wide_sel_envs['rel_to_oxygen'].value_counts()

aerobe                   3940
obligate anaerobe          66
oxic                       59
anaerobe                   29
facultative anaerobes      21
Aerobic                    20
aerobic                    18
anaerobic                  18
Oxic                       13
microaerophilic            11
hypoxic                     6
normal oxic seawater        4
oxic/anoxic boundary        4
Hypoxic                     3
6.0-6.5 mg/l                3
22 mg/l                     3
facultative anaerobe        1
1.0-2.2 mg/l                1
25 mg/l                     1
23.5 mg/l                   1
0 mg/l                      1
aerobic-anaerobic           1
obligate                    1
facultative                 1
Name: rel_to_oxygen, dtype: int64

#### Below I add a new column to harmonized_wide_sel_envs called "rel_to_oxygen_rep_val" in which I put the replaced values. 

I replaced "obligate anaerobe", "facultative anaerobes", "anaerobic", and "facultative anaerobe" all with "anaerobe".
I replaced "obligate anaerobe", "Aerobic" and "aerobic" both with "aerobe".

the rest of the values were unclear of whether they should be replaced with aerobe or anaerobe. 

In [259]:
harmonized_wide_sel_envs['rel_to_oxygen_rep'] = harmonized_wide_sel_envs["rel_to_oxygen"]

harmonized_wide_sel_envs['rel_to_oxygen_rep'] = harmonized_wide_sel_envs['rel_to_oxygen_rep'].replace(to_replace = ["obligate anaerobe", "facultative anaerobes", "anaerobic", "facultative anaerobe"], value="anaerobe")
harmonized_wide_sel_envs['rel_to_oxygen_rep'] = harmonized_wide_sel_envs['rel_to_oxygen_rep'].replace(to_replace = ["Aerobic", "aerobic"], value="aerobe")

In [260]:
[(harmonized_wide_sel_envs["rel_to_oxygen_rep"] != None)]

truefalselist = (harmonized_wide_sel_envs["rel_to_oxygen_rep"] != "aerobe") & (harmonized_wide_sel_envs["rel_to_oxygen_rep"] != "anaerobe") & (harmonized_wide_sel_envs["rel_to_oxygen_rep"].isnull() == False)
harmonized_wide_sel_envs['rel_to_oxygen_rep_status'] = truefalselist

harmonized_wide_sel_envs['rel_to_oxygen_rep_status'] = harmonized_wide_sel_envs['rel_to_oxygen_rep_status'].replace(to_replace = True, value="Couldn't Repair")
harmonized_wide_sel_envs['rel_to_oxygen_rep_status'] = harmonized_wide_sel_envs['rel_to_oxygen_rep_status'].replace(to_replace = False, value="Repaired or Already None")

harmonized_wide_sel_envs[harmonized_wide_sel_envs['rel_to_oxygen_rep_status'] == "Couldn't Repair"]

#SET "Couldn't Repair" VALUES  IN rel_to_oxygen_rep TO None:
harmonized_wide_sel_envs['rel_to_oxygen_rep'] = harmonized_wide_sel_envs['rel_to_oxygen_rep'].replace(to_replace = ['obligate', 'oxic', 'hypoxic', 'aerobic-anaerobic', 'oxic/anoxic boundary', 'microaerophilic', '1.0-2.2 mg/l', '0 mg/l', '6.0-6.5 mg/l', '22 mg/l', '23.5 mg/l', '25 mg/l', 'normal oxic seawater', 'Hypoxic', 'Oxic', 'facultative'], value=None)

#UNCOMMENT BELOW TO SEE COULDNT REPAIR VALUES:
#harmonized_wide_sel_envs[harmonized_wide_sel_envs["rel_to_oxygen_rep_status"] == "Couldn't Repair"].head()

#FINAL DATAFRAME
harmonized_wide_sel_envs.head()

,raw_id,agrochem_addition,air_temp,air_temp_regm,altitude,bacteria_carb_prod,barometric_press,chem_administration,chloride,collection_date,collection_device,collection_method,cur_land_use,cur_vegetation,cur_vegetation_meth,depth,description,diss_carb_dioxide,diss_inorg_nitro,diss_inorg_phosp,diss_org_nitro,drainage_class,elev,env_broad_scale,env_local_scale,env_medium,env_package,experimental_factor,extreme_event,extreme_salinity,fao_class,filter_type,genotype,geo_loc_exposure,geo_loc_name,growth_med,heavy_metals,heavy_metals_meth,height_or_length,horizon,horizon_meth,host_taxid,humidity,humidity_regm,lat_lon,link_addit_analys,misc_param,nitrate,nitrite,nitro,oxy_stat_samp,pesticide_regm,ph,ph_meth,ph_regm,phosphate,plant_product,previous_land_use,previous_land_use_meth,rel_to_oxygen,salinity,salinity_meth,same_as,samp_collect_device,samp_salinity,samp_store_loc,samp_store_temp,sample_name,slope_aspect,slope_gradient,soil_type,soil_type_meth,soluble_react_phosp,source_material_id,source_name,submitted_sample_id,submitted_subject_id,temp,tillage,tissue,tot_phosp,tot_phosphate,water_content_soil,water_content_soil_meth,wind_direction,wind_speed,rel_to_oxygen_rep,rel_to_oxygen_rep_status
0,26206,None,None,None,None,None,None,None,None,2002-11-02,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,water,None,None,None,None,None,None,None,Spain,None,None,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",None,None,None,None,None,None,37.72498333 N 6.56025 W,None,None,None,None,None,None,None,2.28,None,None,None,None,None,None,None,None,None,None,Bottle,None,None,None,AG11,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Repaired or Already None
1,26207,None,None,None,None,None,None,None,None,2002-11-02,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,water,None,None,None,None,None,None,None,Spain,None,None,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",None,None,None,None,None,None,37.72498333 N 6.56025 W,None,None,None,None,None,None,None,2.28,None,None,None,None,None,None,None,None,None,None,Bottle,None,None,None,AG12,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Repaired or Already None
2,26208,None,None,None,None,None,None,None,None,2002-11-02,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,water,None,None,None,None,None,None,None,Spain,None,None,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",None,None,None,None,None,None,37.72498333 N 6.56025 W,None,None,None,None,None,None,None,2.28,None,None,None,None,None,None,None,None,None,None,Bottle,None,None,None,AG13,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Repaired or Already None
3,26209,None,None,None,None,None,None,None,None,2002-11-02,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,water,None,None,None,None,None,None,None,Spain,None,None,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",None,None,None,None,None,None,37.7249 N 6.5625 W,None,None,None,None,None,None,None,1.95,None,None,None,None,None,None,None,None,None,None,Bottle,None,None,None,AG21,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Repaired or Already None
4,26210,None,None,None,None,None,None,None,None,2002-11-02,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,water,None,None,None,None,None,None,None,Spain,None,None,"Palacios C, Zettler E, Amils R, Amaral-Zettler...",None,None,None,None,None,None,37.7249 N 6.5625 W,None,None,None,None,None,None,None,1.97,None,None,None,None,None,None,None,None,None,None,Bottle,None,None,None,AG22,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Repaired or Already None


In [261]:
harmonized_wide_sel_envs["rel_to_oxygen_rep"].unique()

array([None, 'anaerobe', 'aerobe'], dtype=object)

In [262]:
harmonized_wide_sel_envs["rel_to_oxygen_rep"].value_counts()

aerobe      3979
anaerobe     135
Name: rel_to_oxygen_rep, dtype: int64

In [263]:
len(harmonized_wide_sel_envs)

48892